In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense

import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [3]:
train_x = np.load('/home/ckwwk/data/germination_data/train_germination.npy')
val_x = np.load('/home/ckwwk/data/germination_data/val_germination.npy')
test_x = np.load('/home/ckwwk/data/germination_data/test_germination.npy')

In [4]:
t1 = train_x[:,:,:,80:140]

In [5]:
t1.shape

(1000, 61, 61, 60, 1)

In [6]:
v1 = val_x[:,:,:,80:140]

In [7]:
v1.shape

(250, 61, 61, 60, 1)

In [8]:
tt1 = test_x[:,:,:,80:140]

In [9]:
tt1.shape

(286, 61, 61, 60, 1)

In [10]:
import pandas as pd

label_df = pd.read_csv("/home/ckwwk/data/label/germination_label.csv")
label_df.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [11]:
from keras.utils import to_categorical
train_y = label_df[0:1000]
train_y = to_categorical(train_y)

In [12]:
val_y = label_df[1000:1250]
val_y = to_categorical(val_y)
test_y = label_df[1250:1536]
test_y = to_categorical(test_y)

In [13]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [14]:
input_layer = Input((61, 61, 60,1))

conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 5), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)

print(conv_layer4._keras_shape)
conv3d_shape = conv_layer4._keras_shape
conv_layer4 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer4)
conv_layer5 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer4)

flatten_layer = Flatten()(conv_layer5)

dense_layer1 = Dense(units=1024, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=2, activation='softmax')(dense_layer2)

W0809 15:48:26.039416 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 15:48:26.041444 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 15:48:26.044916 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 15:48:26.149493 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.com

(None, 53, 53, 48, 64)


In [15]:
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 61, 61, 60, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 59, 59, 56, 8)     368       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 57, 57, 52, 16)    5776      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 55, 55, 50, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 53, 53, 48, 64)    55360     
_________________________________________________________________
reshape_1 (Reshape)          (None, 53, 53, 3072)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 51, 51, 64)        1769536   
__________

In [16]:
#adam = Adam(lr=0.005, decay=1e-06)
#model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

W0809 15:48:26.252736 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0809 15:48:26.259585 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [17]:
filepath = "best-model_V7_1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = keras.callbacks.TensorBoard(log_dir='/home/ckwwk/data/C2/tensor_log_V7_1',histogram_freq=1,)
callbacks_list = [checkpoint,tb]

In [ ]:
history = model.fit(x=t1, y=train_y, batch_size=32, epochs=1000, validation_data=(v1,val_y),callbacks=callbacks_list)

W0809 15:48:38.005628 140073036056384 deprecation.py:323] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 250 samples


W0809 15:48:43.687303 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:848: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0809 15:48:43.732171 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0809 15:48:43.735176 140073036056384 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/1000
1000/1000 [==============================] - 39s 39ms/step - loss: 0.6685 - acc: 0.6210 - val_loss: 0.6824 - val_acc: 0.6160

Epoch 00001: val_acc improved from -inf to 0.61600, saving model to best-model_V7_1.hdf5
Epoch 2/1000
1000/1000 [==============================] - 26s 26ms/step - loss: 0.6622 - acc: 0.6320 - val_loss: 0.6658 - val_acc: 0.6160

Epoch 00002: val_acc did not improve from 0.61600
Epoch 3/1000
1000/1000 [==============================] - 26s 26ms/step - loss: 0.6646 - acc: 0.6320 - val_loss: 0.7334 - val_acc: 0.6160

Epoch 00003: val_acc did not improve from 0.61600
Epoch 4/1000
1000/1000 [==============================] - 26s 26ms/step - loss: 0.6644 - acc: 0.6320 - val_loss: 0.6681 - val_acc: 0.6160

Epoch 00004: val_acc did not improve from 0.61600
Epoch 5/1000
1000/1000 [==============================] - 25s 25ms/step - loss: 0.6620 - acc: 0.6320 - val_loss: 0.6668 - val_acc: 0.6160

Epoch 00005: val_acc did not improve from 0.61600
Epoch 6/1000
100

In [ ]:
model.load_weights("best-model_V7_1.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.evaluate(tt1,test_y,batch_size=32)